In [1]:
import pandas as pd
import numpy as np
#import get_prices as hist
import tensorflow as tf
from preprocessing import DataProcessing as process
import pandas_datareader.data as pdr
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.model_selection import train_test_split

In [2]:
palu=pd.read_excel('C:\\Users\\ASUS\\Documents\\these\\code_these-master\\palu_impute001.xlsx')
paluu=pd.DataFrame(palu, columns = ['PPOIDS', 'TEMPERATURE', 'S_M8_APPETIT', 'S_FATIGUE', 'S_ARTHRALGI',
                                    'S_T_DIGESTIF', 'S_VERTIGE', 'S_FRISSON', 'S_MYALGIE', 'S_DABDO', 
                                    'S_VOMISS', 'S_NAUSEE', 'S_CEPHALE', 'S_FIEVRE','TDR'])

# Nous séparons notre variable cible en deux parties. Une partie pour entrainer y_train et une partie pour tester y_test
X = paluu
y = palu['Diagnostic']
X1_train, MX_test, y1_train, My_test = train_test_split(X, y, random_state=0)

#Suréchantillonnage avec l'algorithme SMOTE
X1 = paluu
y1 =palu['Diagnostic']
y1= pd.DataFrame(y1)


from imblearn.over_sampling import SMOTE
Re= SMOTE(random_state=0)
#X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=0)
#columns = X1_train.columns
Re_data_X1,Re_data_y1=Re.fit_sample(X1_train, y1_train)
Re_data_X1 = pd.DataFrame(data=Re_data_X1 )


# Nous séparons notre jeu données palu en deux parties. Une partie pour entrainer X_train et un partie pour tester X_test
# Nous séparons notre variable cible en deux parties. Une partie pour entrainer y_train et une partie pour tester y_test
X_train =Re_data_X1
y_train=Re_data_y1
#y_train= pd.DataFrame(Re_data_y1)

Using TensorFlow backend.


In [3]:
X_train.shape

(30698, 15)

In [4]:
X_train.to_csv('pal.csv')

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy

In [6]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=15, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
#dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = X_train
Y = y_train
# create model
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.9170304536819458


In [7]:
from sklearn.model_selection import GridSearchCV
import numpy
 
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=15, kernel_initializer=init, activation='relu'))
    model.add(Dense(15, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
 
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
#dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = X_train
Y = y_train
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [2, 4, 6]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.919506 using {'batch_size': 10, 'epochs': 6, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.888201 (0.037416) with: {'batch_size': 5, 'epochs': 2, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.883738 (0.028970) with: {'batch_size': 5, 'epochs': 2, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.909799 (0.056381) with: {'batch_size': 5, 'epochs': 2, 'init': 'normal', 'optimizer': 'rmsprop'}
0.882370 (0.037213) with: {'batch_size': 5, 'epochs': 2, 'init': 'normal', 'optimizer': 'adam'}
0.897257 (0.061869) with: {'batch_size': 5, 'epochs': 2, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.915793 (0.061191) with: {'batch_size': 5, 'epochs': 2, 'init': 'uniform', 'optimizer': 'adam'}
0.885856 (0.036036) with: {'batch_size': 5, 'epochs': 4, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.890025 (0.030043) with: {'batch_size': 5, 'epochs': 4, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.909115 (0.050974) with: {'batch_size': 5, 'epochs': 4, 'init': 'normal', 'optimizer': 'r